# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key - (Not importing the key because I don't feel comfortable giving my credit card info for the API key usage to google.)
# from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
csv_file = os.path.join('/Users/jacobcortez/Documents/GitHub/DS_Repos/HW_Repo/python_api_challenge/weather_py/weather_df.csv')

weather_df2 = pd.read_csv(csv_file)

weather_df2.head()


,City ID,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,2126199,Cherskiy,RU,68.7500,68.7500,-6.03,100,96,3.51
1,3372760,Praia da Vitória,PT,38.7333,38.7333,66.45,82,75,10.83
2,3380290,Sinnamary,GF,5.3833,5.3833,78.44,88,100,11.95
3,1507390,Dikson,RU,73.5069,73.5069,8.06,87,99,12.55
4,2122090,Pevek,RU,69.7008,69.7008,10.90,93,100,7.14


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Configure gmaps
# gmaps.configure(api_key=g_key)

In [21]:
# Create heatmap
locations = weather_df2[['Latitude', 'Longitude']]
humidity = weather_df2['Humidity']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 10


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
ideal_city_df = weather_df2[(weather_df2['Temperature'] >= 75) & (weather_df2['Temperature'] <= 85)]
ideal_city_df = ideal_city_df[ideal_city_df['Wind Speed'] <= 5]
ideal_city_df = ideal_city_df[ideal_city_df['Cloudiness'] <= 5]
ideal_city_df = ideal_city_df[ideal_city_df['Humidity'] <= 70]
hotel_df = ideal_city_df
hotel_df

,City ID,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
232,287286,Muscat,OM,23.6139,23.6139,80.62,50,0,3.13
348,4449620,Vicksburg,US,32.3526,32.3526,79.21,65,1,4.61
408,104515,Makkah al Mukarramah,SA,21.4267,21.4267,80.17,59,0,0.31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
# Add a column to the Hotel DF
hotel_df['Hotel Name'] = ''
hotel_df

,City ID,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
232,287286,Muscat,OM,23.6139,23.6139,80.62,50,0,3.13,
348,4449620,Vicksburg,US,32.3526,32.3526,79.21,65,1,4.61,
408,104515,Makkah al Mukarramah,SA,21.4267,21.4267,80.17,59,0,0.31,


In [27]:
# Add hotel names to column based on parameters

for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {"keyword": "hotel", "radius": 5000,"key": #g_key,}

        Latitude = row['Latitude']
        Longitude = row['Longitude']

        params['location'] = f"{Latitude}, {Longitude}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        

SyntaxError: invalid syntax (<ipython-input-27-45f3af219f25>, line 10)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure
